In [1]:

import numpy as np
import pandas as pd

###################################
# 1. READ CSV AND PREPARE DATA
###################################
# Make sure to replace "data_file.csv" with your actual CSV file path.
# We'll handle it with a try/except to allow a dummy fallback.

try:
    df = pd.read_csv("ScanRecords.csv")
    # Drop rows where 'Duration' is missing or placeholder (e.g. "########")
    df = df.dropna(subset=["Duration"])
    df = df[df["Duration"] != "########"]
    df["Duration"] = pd.to_numeric(df["Duration"], errors="coerce")
    df = df.dropna(subset=["Duration"])  # final cleanup
except:
    # Dummy fallback if file isn't found/parsable
    dummy_data = {
        "Date": ["2024-11-01", "2024-11-01", "2024-11-02", "2024-11-02"],
        "Time": [9.5, 10.2, 8.2, 16.5],
        "Duration": [0.7, 0.9, 1.2, 1.05],  # in hours
        "PatientType": ["Type 2", "Type 2", "Type 2", "Type 2"]
    }
    df = pd.DataFrame(dummy_data)

# Separate out the Type 2 data (for non-parametric sampling)
df_type2 = df[df["PatientType"] == "Type 2"].copy()

# We can define "daily" arrivals by grouping if you want day-block bootstrap
df_type2_daily = df_type2.groupby("Date")["Time"].count().reset_index(name="Arrivals")
durations_type2_data = df_type2["Duration"].values  # hours for Type 2

###################################
# 2. PART 1 RESULTS (INPUTS)
###################################
# --- Type 1 INTER-ARRIVAL TIME ---
#  You have a 95% CI for the time between calls of about [29.78, 36.32] minutes.
#  This suggests that the *mean* inter-arrival time is ~ 33 minutes (the midpoint), 
#  or you might treat it as uncertain in that range.

INTER_ARRIVAL_T1 = 33.0  # minutes, as a rough midpoint
# Convert to an exponential rate (calls per minute):
RATE_T1 = 1.0 / INTER_ARRIVAL_T1  # ~ 0.0303 calls/min

# If you prefer to sample the *mean* from that 95% CI each simulation,
# you could do something like:
# INTER_ARRIVAL_T1 = np.random.uniform(29.78, 36.32)
# RATE_T1 = 1.0 / INTER_ARRIVAL_T1

# --- Type 1 SCAN DURATION (Normal approx) ---
# From your Part 1 results, say mean ~ 0.43 hours, std ~ 0.095 hours
MEAN_DUR_T1 = 0.43  # hours
STD_DUR_T1  = 0.095 # hours

# --- Type 2 ARRIVALS (non-parametric daily sampling) ---
type2_daily_counts = df_type2_daily["Arrivals"].values  # for day-block bootstrap

# --- Type 2 SCAN DURATIONS (non-parametric) ---
# We'll just re-sample from durations_type2_data
# (which are in hours).

###################################
# 3. SIMULATION CONFIG
###################################
NUM_DAYS = 20      # total days to simulate
OPEN_TIME = 8.0    # 8:00
CLOSE_TIME = 17.0  # 17:00
HOURS_PER_DAY = CLOSE_TIME - OPEN_TIME  # 9
MINUTES_PER_DAY = HOURS_PER_DAY * 60    # 540

# Timeslot approach (if the hospital wants fixed scheduling blocks)
TIMESLOT_T1 = 30  # minutes
TIMESLOT_T2 = 60  # minutes

###################################
# 4. HELPER FUNCTIONS
###################################
def generate_type1_arrivals_exponential():
    """
    Generate Type 1 arrivals *throughout the workday (8-17)* 
    by sampling inter-arrival times from Exp(RATE_T1).
    We'll keep generating arrivals until we exceed the day length in minutes (540).
    Then we draw each patient's SCAN DURATION from Normal(mean, std).
    Returns a list of (arrival_time_in_minutes_from_8, duration_in_minutes).
    """
    arrivals = []
    current_min = 0.0
    while current_min < MINUTES_PER_DAY:
        # Sample next inter-arrival from Exponential( RATE_T1 )
        delta = np.random.exponential(1.0/RATE_T1)
        current_min += delta
        if current_min >= MINUTES_PER_DAY:
            break
        # Draw scan duration (in hours -> minutes)
        dur = np.random.normal(MEAN_DUR_T1, STD_DUR_T1)
        dur_in_min = max(0.0, dur*60.0)
        arrivals.append((current_min, dur_in_min))
    return arrivals

def generate_type2_arrivals_daily():
    """
    Day-block bootstrap for the *number of Type 2 arrivals in a day*, 
    then we place them *uniformly* in [0, 540) or keep them "day-based".
    Durations are re-sampled from Type 2 distribution.
    Returns a list of (arrival_time_in_minutes_from_8, duration_in_minutes).
    """
    if len(type2_daily_counts) == 0 or len(durations_type2_data) == 0:
        return []  # if no data
    # sample # arrivals from Type 2 day-block
    boot_idx = np.random.randint(0, len(type2_daily_counts))
    num_arrivals = type2_daily_counts[boot_idx]
    # for each arrival, choose a random arrival_time uniform in [0, 540)
    arrival_times = np.random.uniform(low=0, high=MINUTES_PER_DAY, size=num_arrivals)
    arrival_times.sort()  # if you want them in ascending order
    
    # re-sample durations from historical Type 2 durations
    dur_idx = np.random.randint(0, len(durations_type2_data), size=num_arrivals)
    dur_in_hrs = durations_type2_data[dur_idx]
    dur_in_min = dur_in_hrs * 60.0
    
    # pair them
    arrivals = list(zip(arrival_times, dur_in_min))
    return arrivals

def schedule_old_system(type1_arrivals, type2_arrivals):
    """
    Old system => 2 separate machines:
      * MRI1 for Type 1
      * MRI2 for Type 2
    We assume each arrival is scheduled in the earliest timeslot that doesn't exceed 17:00.
    (Or we allow overtime if it doesn't fit.)
    
    Input: 
      type1_arrivals: list of (arr_time, dur_minutes)
      type2_arrivals: list of (arr_time, dur_minutes)
    Output:
      schedule_mri1: list of (start_min, end_min, real_duration)
      schedule_mri2: list of (start_min, end_min, real_duration)
    """
    # We only need the *order* in which they call if that matters. 
    # For old system, we don't actually have to worry about arrival times except to preserve order.
    # We'll sort by arrival_time if needed:
    type1_arrivals = sorted(type1_arrivals, key=lambda x: x[0])
    type2_arrivals = sorted(type2_arrivals, key=lambda x: x[0])

    # For scheduling, we place them in blocks of TIMESLOT_T1 or TIMESLOT_T2 
    schedule_mri1 = []
    schedule_mri2 = []
    
    current_time_mri1 = 0.0
    for (_, real_dur) in type1_arrivals:
        slot_len = TIMESLOT_T1
        start = current_time_mri1
        end = start + slot_len
        schedule_mri1.append((start, end, real_dur))
        current_time_mri1 = end
    
    current_time_mri2 = 0.0
    for (_, real_dur) in type2_arrivals:
        slot_len = TIMESLOT_T2
        start = current_time_mri2
        end = start + slot_len
        schedule_mri2.append((start, end, real_dur))
        current_time_mri2 = end
    
    return schedule_mri1, schedule_mri2

def schedule_new_system(type1_arrivals, type2_arrivals):
    """
    New system => 2 identical MRI machines, can handle Type 1 or Type 2.
    We'll do a simple "greedy" approach: whichever machine is free first gets the next arrival.
    We'll sort arrivals by arrival_time to simulate the chronological scheduling. 
    Then for each arrival (type 1 or 2), we schedule a block of length = TIMESLOT_T1 or TIMESLOT_T2.
    """
    # Combine arrivals, keep track of which is T1 vs T2
    labeled = [(t, d, 'T1') for (t,d) in type1_arrivals] + [(t, d, 'T2') for (t,d) in type2_arrivals]
    # sort by call time
    labeled.sort(key=lambda x: x[0])
    
    schedA = []
    schedB = []
    currentA = 0.0
    currentB = 0.0
    
    for (call_time, real_dur, ptype) in labeled:
        slot_len = TIMESLOT_T1 if ptype=='T1' else TIMESLOT_T2
        # whichever machine is free sooner:
        if currentA <= currentB:
            start = currentA
            end = start + slot_len
            schedA.append((start, end, real_dur))
            currentA = end
        else:
            start = currentB
            end = start + slot_len
            schedB.append((start, end, real_dur))
            currentB = end
    
    return schedA, schedB

def run_schedule(schedule_mri):
    """
    For a single MRI schedule: list of (start_min, end_min, real_duration).
    - The 'start_min/end_min' is the scheduled slot.
    - 'real_duration' is how long the scan actually takes (could be more/less).
    We'll calculate the real finishing time 
       = start_min + real_duration 
      for each patient, 
    then the overall finishing time is the max of all real finishing times.
    Also compute total busy time for potential utilization.
    """
    finishing_time = 0.0
    busy_time = 0.0
    for (s, e, dur) in schedule_mri:
        real_finish = s + dur  # actual usage
        finishing_time = max(finishing_time, real_finish)
        busy_time += (real_finish - s)
    return finishing_time, busy_time

###################################
# 5. MAIN SIMULATION
###################################
NUM_REPS = 1  # run how many replications?
results_old = []
results_new = []

for rep in range(NUM_REPS):
    daily_finishing_old = []
    daily_overtime_old = []
    daily_finishing_new = []
    daily_overtime_new = []
    
    for day in range(NUM_DAYS):
        # (A) Generate Type 1 arrivals (exponential arrival times over 8-17)
        type1_arrivals = generate_type1_arrivals_exponential()
        # (B) Generate Type 2 arrivals (daily-block approach)
        type2_arrivals = generate_type2_arrivals_daily()
        
        # OLD SYSTEM SCHEDULING
        sched1, sched2 = schedule_old_system(type1_arrivals, type2_arrivals)
        finish1, busy1 = run_schedule(sched1)
        finish2, busy2 = run_schedule(sched2)
        daily_finish_old = max(finish1, finish2)
        overtime_old = max(0, daily_finish_old - MINUTES_PER_DAY)
        daily_finishing_old.append(daily_finish_old)
        daily_overtime_old.append(overtime_old)
        
        # NEW SYSTEM SCHEDULING
        schedA, schedB = schedule_new_system(type1_arrivals, type2_arrivals)
        finishA, busyA = run_schedule(schedA)
        finishB, busyB = run_schedule(schedB)
        daily_finish_new = max(finishA, finishB)
        overtime_new = max(0, daily_finish_new - MINUTES_PER_DAY)
        daily_finishing_new.append(daily_finish_new)
        daily_overtime_new.append(overtime_new)
    
    # Aggregate results for this replication
    avg_finish_old = np.mean(daily_finishing_old)
    avg_ot_old     = np.mean(daily_overtime_old)
    results_old.append((avg_finish_old, avg_ot_old))
    
    avg_finish_new = np.mean(daily_finishing_new)
    avg_ot_new     = np.mean(daily_overtime_new)
    results_new.append((avg_finish_new, avg_ot_new))

###################################
# 6. SUMMARIZE RESULTS
###################################
mean_finish_old, mean_ot_old = np.mean(results_old, axis=0)
mean_finish_new, mean_ot_new = np.mean(results_new, axis=0)

print("====== OLD SYSTEM RESULTS ======")
print(f"Avg finishing time (min from 8:00): {mean_finish_old:.2f}")
print(f"Avg daily overtime (minutes):      {mean_ot_old:.2f}")

print("====== NEW SYSTEM RESULTS ======")
print(f"Avg finishing time (min from 8:00): {mean_finish_new:.2f}")
print(f"Avg daily overtime (minutes):      {mean_ot_new:.2f}")


====== OLD SYSTEM RESULTS ======
Avg finishing time (min from 8:00): 641.09
Avg daily overtime (minutes):      106.86
====== NEW SYSTEM RESULTS ======
Avg finishing time (min from 8:00): 559.87
Avg daily overtime (minutes):      36.44
